In [ ]:
# !git clone https://github.com/Cassini-chris/IEL IEL3

In [ ]:
# Welcome Message
from IPython.display import Image
Image("img/head.png", width=800)

In [ ]:
# Guiding Information
Image("img/task_0.png", width=800)

In [ ]:
# Inspecting the Machine Type and GPU
# -----------------------------------
# Task 1: Install Watermark and retrieve system architecture by running the code below

!pip install watermark 
!pip install watermark[gpu]
%load_ext watermark
%watermark
%watermark --iversions
%watermark --gpu

In [ ]:
# Process architecture
Image("img/task_8a.png", width=800)

In [ ]:
Image("img/task_8b.png", width=800)

In [ ]:
# Boostrapping the environment 
Image("img/task_1.png", width=800)

In [ ]:
# Boostrapping the environment
# ----------------------------
# Task 2: Run the cell and fill in the parameter values

project = !gcloud config get-value project
PROJECT_ID = project[0]
REGION = input('Enter the name of the Region: ')
GCS_BUCKET = input('Enter the name of the GCS bucket: ')
source_document = input('Enter the name of the source document: ')
print ("Your PROJECT_ID is: ", PROJECT_ID)


### ANSWER ############################################################
# project = !gcloud config get-value project
# PROJECT_ID = project[0]
# REGION = 'us-central1'
# GCS_BUCKET = 'cymbalstay-docs-iel-genai-dryrun'
# source_document = 'full_cymbal_stay.pdf'
# PROJECT_ID
#######################################################################

In [ ]:
# Install dependencies
Image("img/task_3.png", width=800)

In [ ]:
# Install dependencies
# -----------------------------------
# Task 3: Find the Packages on PyPi and install them accordingly using "!pip install"
# URL: https://pypi.org/

!pip install "Package Name: AI Platform"
!pip install "Package Name: DocAI"
!pip install "Package Name: ScaNN"
!pip install "Package Name: PyPDF"

print('Installation completed!')

### ANSWER ############################################################
# !pip install google.cloud.aiplatform
# !pip install google-cloud-documentai
# !pip install scann
# !pip install PyPDF2
#######################################################################

In [ ]:
# Importing packages
# -----------------------------------
# Task 4: Just run this cell. There is no todo!
import os
import io
import json
import base64
import requests
import concurrent.futures
import time
import PyPDF2
import IPython
import PIL
import PIL.ImageFont, PIL.Image, PIL.ImageDraw
import shapely
import scann
import numpy as np

import vertexai.preview.language_models
from google.cloud import aiplatform
from google.cloud import documentai
from google.cloud.documentai_v1 import Document
from google.cloud import storage

print("AI-Platform Version. Make sure version is above 1.30. Your Version: ", aiplatform.__version__)

In [ ]:
# Initiate VertexAI | DocAI clients 
Image("img/task_5.png", width=800)

In [ ]:
# Initiate VertexAI clients 
# -----------------------------------
# Task 5a: Just run this cell. There is no todo!

vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)

print("Full Region name: " , REGION)

In [ ]:
# Initiate Document AI client
# -----------------------------------
# Task 5b: Add your Region Prefix to the DocAI API Endpoint via client_options
# For example: us-central1-a = us | europe-west6-c = europe

# Set Region Prefix
REGION_PREFIX = "Add Region Prefix here"
print(REGION_PREFIX)

### ANSWER ############################################################
# REGION_PREFIX = REGION.split('-')[0]
# print(REGION_PREFIX)
######################################################################

docai_client = documentai.DocumentProcessorServiceClient(
    client_options = dict(api_endpoint = f"{REGION_PREFIX}-documentai.googleapis.com")
)

In [ ]:
# Load Models
Image("img/task_6.png", width=800)

In [ ]:
# Load Models
# -----------------------------------
# Task 6: Go to https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models
# Find the latest text model and the embeedings model from June 2023 and add them to the code below:

embedding_model = vertexai.preview.language_models.TextEmbeddingModel.from_pretrained('LATEST Embedding Model')
textgen_model = vertexai.preview.language_models.TextGenerationModel.from_pretrained('USE Text Model Version from June 2023')

### ANSWER ############################################################
# embedding_model = vertexai.preview.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko@001')
# textgen_model = vertexai.preview.language_models.TextGenerationModel.from_pretrained('text-bison@001')
######################################################################

In [ ]:
# Helper Functions
Image("img/task_7.png", width=800)

In [ ]:
# Extract_pdf_text
# -----------------------------------
# Task 7: Go to https://cloud.google.com/document-ai/docs/form-parser
# Find the latest form parser and add it to the Document AI processor below in the variable 'PARSER_VERSION':

def extract_pdf_text():
    """
    Extracts text from PDF documents using Google Cloud Document AI.

    Args:
        source_document (str): The path to the PDF document to extract text from.

    Returns:
        list: A list of dictionaries, where each dictionary represents a paragraph or table in the PDF document.
        Each dictionary contains the following keys:
            * page_content (str): The text of the paragraph or table.
            * metadata (dict): Metadata about the paragraph or table, including the page number, paragraph or table number,
            and the VME ID (a unique identifier for the paragraph or table).
            * extras (dict): Optional additional information about the paragraph or table, such as the vertices of the
            bounding box.
    """
    # Initialize Google Cloud Storage client
    gcs = storage.Client(project=PROJECT_ID)

    # Get the PDF document from the specified bucket
    bucket = gcs.bucket(GCS_BUCKET)
    blob = bucket.blob(source_document)
    response = blob.download_as_bytes()

    # Read the PDF document as a PDF object
    pdf = PyPDF2.PdfReader(io.BytesIO(response))
  
     # Extract text from each page of the PDF document
    pdfs = []
    for page_num, page in enumerate(pdf.pages, 1):
        # Create a new PDFWriter object
        writer = PyPDF2.PdfWriter()

        # Add the current page to the PDFWriter object
        writer.add_page(page)

        # Write the contents of the PDFWriter object to a BytesIO object
        with io.BytesIO() as bytes_stream:
                pdfs.append(writer.write(bytes_stream)[1].getbuffer().tobytes())

                
     # Initialize Google Cloud Document AI client
    docfai_client = documentai.DocumentProcessorServiceClient()

    # Define parameters for Document AI processor
    PARSER_DISPLAY_NAME = "my_general_processor"
    PARSER_TYPE = "FORM_PARSER_PROCESSOR"
    
    PARSER_VERSION = "---- ADD DOCUMENT PARSER HERE ---- "
        
    ### ANSWER ############################################################
    # PARSER_VERSION = "pretrained-form-parser-v2.1-2023-06-26" 
    ######################################################################   
   

    # Get the Document AI processor
    parser = None
    for p in docai_client.list_processors(parent=f"projects/{PROJECT_ID}/locations/{REGION_PREFIX}"):
        if p.display_name == PARSER_DISPLAY_NAME:
            parser = p
            break               
                
    if parser is None:
        # Create a new Document AI processor if it doesn't exist
        parser = docai_client.create_processor(
            parent=f"projects/{PROJECT_ID}/locations/{REGION_PREFIX}",
            processor=dict(
                display_name=PARSER_DISPLAY_NAME, type_=PARSER_TYPE, default_processor_version=PARSER_VERSION
            ),
        )
        
    # Set the rate limit for processing PDF documents
    rate_limit_minute = 120
    adjust_rate_limit = rate_limit_minute / 2
    

    def docai_runner(p, start, raw_document):
      process_options = documentai.ProcessOptions(
      individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
          pages=[1] # e.g. in case you only want to process individual pages
      )
      )
      request = documentai.ProcessRequest(
            name=parser.name,
            raw_document=raw_document,
            process_options=process_options,
        )     
      sleep_time = (p * (60/adjust_rate_limit)) - (time.time() - start)
      if sleep_time > 0: time.sleep(sleep_time)
      return (p, docai_client.process_document(request = dict(raw_document = raw_document, name = parser.name)))
    print(f"The expected runtime for PDF text extraction is {(len(pdfs)/adjust_rate_limit):.2f} minutes")
    results = [None] * len(pdfs)
    start = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(pdfs)) as executor:
        futures = [
            executor.submit(
                docai_runner,
                p, start,
                documentai.RawDocument(content = pdf, mime_type = 'application/pdf')
            ) for p, pdf in enumerate(pdfs)
        ]
        for future in concurrent.futures.as_completed(futures):
          results[future.result()[0]] = (Document.to_dict(future.result()[1].document))
    len(pdfs), len(results)
    documents = []
    page_images = []
    for r, result in enumerate(results):
        results[r]['metadata'] = dict(vme_id = str(r))
        document_image = PIL.Image.open(
            io.BytesIO(
                base64.decodebytes(result['pages'][0]['image']['content'].encode('utf-8'))
                )
            )
        page_images.append(document_image)
        tables = []
        for t, table in enumerate(result['pages'][0]['tables']):
          table_txt = ''
          if 'text_anchor' in table['layout'].keys():
            for s, segment in enumerate(table['layout']['text_anchor']['text_segments']):
              if t == 0 and s == 0: start = 0
              else: start = int(segment['start_index'])
              end = int(segment['end_index'])
              table_txt += result['text'][start:end+t]
              # print(table_txt)
          documents.append(
              dict(
                  page_content = table_txt,
                  metadata = dict(
                      page = r+1,
                      table = t+1,
                      vme_id = str(len(documents)),
                      filename = source_document.split('/')[-1],
                      source_document = source_document
                  ),
                  extras = dict(
                      vertices = vertices
                  )
              )
          )
        for p, paragraph in enumerate(result['pages'][0]['paragraphs']):
          paragraph_txt = ''
          for s, segment in enumerate(paragraph['layout']['text_anchor']['text_segments']):
            if p == 0 and s == 0: start = 0
            else: start = int(segment['start_index'])
            end = int(segment['end_index'])
            paragraph_txt += result['text'][start:end+1]
            #print(paragraph_txt)
          vertices = []
          for vertex in paragraph['layout']['bounding_poly']['normalized_vertices']:
            vertices.append(dict(x = vertex['x'] * document_image.size[0], y = vertex['y'] * document_image.size[1]))
          use_paragraph = True
          for t_shape in tables:
            p_shape = shapely.geometry.Polygon([(v['x'], v['y']) for v in vertices])
            if p_shape.intersects(t_shape): use_paragraph = False
          if use_paragraph:
            documents.append(
                dict(
                    page_content = paragraph_txt,
                    metadata = dict(
                        page = r+1,
                        paragraph = p+1,
                        vme_id = str(len(documents)),
                        filename = source_document.split('/')[-1],
                        source_document = source_document
                    ),
                    extras = dict(
                        vertices = vertices
                    )
                )
            )
    print(f"Finished processing of {len(documents)} documents")
    return documents

In [ ]:
# Create an embedding_space
# -----------------------------------
# Task 8: Let's discuss an Embedding Space
Image("img/task_9.png", width=800)


In [ ]:
Image("img/task_9b.png", width=800)

In [ ]:
Image("img/task_10.png", width=800)

In [ ]:
def create_embedding_space(documents):
    """
    Creates an embedding space for a list of documents.

    Args:
        documents (list): A list of documents.

    Returns:
        scann.ScannTree: The embedding space.
    """

    # Set the rate limit for processing documents.
    rate_limit_minute = 150

    # Print an estimated runtime for embedding space creation.
    print(f"The expected run time for embedding space creation is {(len(documents)*4/rate_limit_minute):.2f} minutes")

    # Start a timer to track the processing time.
    start = time.time()

    # Process each document and extract its embedding.
    for d, document in enumerate(documents):
        if d % rate_limit_minute == 0:
            # Sleep to avoid exceeding the rate limit.
            time.sleep(((time.time() - start) % 60) + 10)
            start = time.time()

        # Get the text from the document.
        text = document['page_content']

        # Generate an embedding for the text if it exists.
        if text:
            embed = embedding_model.get_embeddings([text])[0].values
        else:
            embed = []

        # Store the embedding in the document.
        documents[d]['embedding'] = embed

    # Create an empty index to store the embeddings.
    index = np.empty((len(documents), len(documents[0]['embedding'])))

    # Check if the embeddings are lists or numpy arrays.
    if type(documents[0]['embedding']) == list:
        # Fill the index with the embeddings.
        for i in range(index.shape[0]):
            if documents[i]['page_content']:
                index[i] = documents[i]['embedding']

    # Normalize the index.
    normalized_index = index / np.linalg.norm(index, axis=1)[:, np.newaxis]

    # Create a builder for ScaNN.
    builder = scann.scann_ops_pybind.builder(
        normalized_index,  # index
        10,  # num_neighbors
        "dot_product",  # distance_measure
    )

    # Build the embedding space.
    embedding_space = builder.tree(
        num_leaves=index.shape[0],  # num_leaves
        num_leaves_to_search=index.shape[0],  # num_leaves_to_search
        training_sample_size=index.shape[0]
    ).score_ah(
        2,
        anisotropic_quantization_threshold=0.2
    ).reorder(
        index.shape[0]
    ).build()

    # Print a message indicating that the embedding space has been created.
    print("Finished creating embedding space")

    # Return the embedding space.
    return embedding_space

In [ ]:
# Create an embedding_space
# -----------------------------------
# Task 9: Let's discuss ScaNN
Image("img/task_9c.png", width=800)

In [ ]:
def search_index(embedding_space, query, k):
    """
    Search for the nearest neighbors of a query in an embedding space.

    Args:
        embedding_space: An embedding space object.
        query: A query vector.
        k: The number of nearest neighbors to return.

    Returns:
        A list of (index, distance) pairs, where `index` is the index of a neighbor in the embedding space
        and `distance` is the distance between the query and the neighbor.
    """
    query = embedding_model.get_embeddings([query])[0].values
    neighbors, distances = embedding_space.search(query, final_num_neighbors=k)
    return list(zip(neighbors, distances))



In [ ]:
def create_context(question, embedding_space):
    """
    Create a context for a question by finding the nearest neighbors in an embedding space and base on this retrieving the original document.

    Args:
        question: A question string.
        embedding_space: An embedding space object.

    Returns:
        A string containing the context for the question.
    """

    # Get the score for the closest match
    score = search_index(embedding_space, question, k=1)[0][1]

    # Retrieve related documents - the number is based on the distance
    # score from the closest match
    relevant_documentation = search_index(embedding_space, question, k=1 + 2 * int(10 * (1 - score)))

    # Create the context
    context = []
    for c, doc in enumerate(relevant_documentation):
        context.append(f'Context {c + 1}:\n' + documents[doc[0]]['page_content'])

    return "\n".join(context)

In [ ]:
def create_prompt(question, context, instructions):
    """
    Create a prompt for a question by formatting the context and instructions to the original prompt.

    Args:
        question: A question string.
        context: A string containing the context for the question.
        instructions: A list of instruction strings.

    Returns:
        A string containing the formatted prompt.
    """

    return f"""
        Give a detailed answer to the question using information from the provided contexts.
        {context}
        Also include these Instructions:
        {" ".join(instructions)}

        Question:
        {question}

        Answer and Explanation:
    """


In [ ]:
def document_bot(prompt, parameters):
    """
    Generate a response to a question and provides potential sources for the answer.

    Args:
        prompt: A prompt string.
        parameters: A dictionary of parameters to pass to the text generation model.

    Returns:
        None.
    """

    # Get the score for the closest match
    score = search_index(embedding_space, prompt, k=1)[0][1]

    # Retrieve related documents - the number is based on the distance score from the closest match
    relevant_documentation = search_index(embedding_space, prompt, k=1 + 2 * int(10 * (1 - score)))

    # Prepare context for the prompt
    context = "\n".join(
        [f'Context {c + 1}:\n' + documents[doc[0]]['page_content'] for c, doc in enumerate(relevant_documentation)])

    # Construct the prompt
    modified_prompt = f"""
        Give a detailed answer to the question using information from the provided contexts.
        {context}
        Also include these Instructions:
        {". ".join(instructions)}

        Question:
        {question}

        Answer and Explanation:
    """

    # Generate a response
    response = textgen_model.predict(modified_prompt, **parameters)

    # Get the closest document to the response
    likely_source = search_index(embedding_space, response.text, k=1)[0]

    # Declare the likely source: if the closest document to the response was in the context, pick it, otherwise pick the first context match
    if likely_source[0] in [rd[0] for rd in relevant_documentation]:
        likely = True
        likely_document = documents[likely_source[0]]
    else:
        likely = False
        likely_document = documents[relevant_documentation[0][0]]

    # Format the sources
    sources = "\n".join(
        f"* {documents[doc[0]]['metadata']['source_document']}#page={documents[doc[0]]['metadata']['page']}\n\t* Document: {documents[doc[0]]['metadata']['filename']}, page: {documents[doc[0]]['metadata']['page']}, relevance to question: {doc[1]:.2f}"
        for doc in relevant_documentation)

    # Format the answer
    answer = f"""---
**Search results** 
Score: {score} 
Closest document: {likely_source} 
"""

    if likely:
        answer = answer + f"Document used to generate response: {likely_document['metadata']['filename']}, page: {likely_document['metadata']['page']}"
    else:
        answer = answer + f"First relevant document used to generate response: {likely_document['metadata']['filename']}, page: {likely_document['metadata']['page']}"

    answer = answer + f"""


{question}

**Answer**
{response.text}

**Potential Sources**  
{sources}
    """

    # Display the answer
    IPython.display.display(IPython.display.Markdown(answer))

# Extract text from PDF document in GCS

In [ ]:
documents = extract_pdf_text()

# Create embedding space

In [ ]:
embedding_space = create_embedding_space(documents)

# Define a Help Center question

In [ ]:
question = "What is the refund policy?"

# Example Questions:
#question = "What are essential amenities for guests?"
#question = "How much is the minimum cancellation fee for Hosts on Cymbal for a confirmed reservation?"
#question = "How old is Barack Obama?"
#question = "Is it a nice time travelling to Sydney in June?"

# Create context to be used in LLM prompt

In [ ]:
context = create_context(question, embedding_space)
context

# Create instructions to be used in LLM prompt

In [ ]:
instructions = [
    "If the question can be answered by the context provided above alone than only use the context to provide the answer. However, never use other information than from the context above to formulate the answer",
    "If the question cannot be answered by the context provided above but is a travel question then use the underlying LLM to answer. In particular state: the context above is not sufficient to answer the question but this is what I found in the base-model"
]
". ".join(instructions)

# Create LLM prompt

In [ ]:
prompt = create_prompt(question, context, instructions)
prompt

# Define prediction parameters

In [ ]:
parameters = {
    "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
    "temperature": 0.7,  # Temperature controls the degree of randomness in token selection.
    "max_output_tokens": 1000,  # Token limit determines the maximum amount of text output.
}

# Ask the Help Center question

In [ ]:
document_bot(prompt, parameters)

# Ask a travel question

In [ ]:
question = "Is it a nice time to travel to Sydney in June?"
context = create_context(question, embedding_space)
prompt = create_prompt(question, context, instructions)
document_bot(prompt, parameters)

# Ask a non-travel question

In [ ]:
question = "Who is Barack Obama?"
context = create_context(question, embedding_space)
prompt = create_prompt(question, context, instructions)
document_bot(prompt, parameters)

# Change instructions to restrict to travel questions

In [ ]:
instructions = [
    "If the question can be answered by the context provided above alone than only use the context to provide the answer. However, never use other information than from the context above to formulate the answer",
    "If the prompt does not contain a travel specific question then do not contiune. Instead end the conversation with: This is not a travel question, sorry I cannot answer this",
    "If the question cannot be answered by the context provided above but is a travel question then use the underlying LLM to answer. In particular state: the context above is not sufficient to answer the question but this is what I found in the base-model"
]
". ".join(instructions)

# Ask the non-travel related question again

In [ ]:
question = "Who is Barack Obama?"
context = create_context(question, embedding_space)
prompt = create_prompt(question, context, instructions)
document_bot(prompt, parameters)